In [42]:
import io
import numpy as np
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types
import sys

In [54]:
def detect_text(uri):
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    client = vision.ImageAnnotatorClient()
    image = types.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    
    orig_stdout = sys.stdout
    f = open('out.txt','w')
    sys.stdout = f
    
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))
    
    sys.stdout = orig_stdout
    
    return




In [10]:
import boto
from boto.s3.key import Key

def readImagefromS3(imageFile):
    """
        This function will be using the pem file for AWS. 
        It will access a public S3 bucket called dvidr with
        the specified image file. The output is a url link
        for the image that will be passed onto the Google
        Cloud Vision API for processing.
        
    """
    with open('../../../dvidr.pem') as f:
        keys = f.read().split(',')

    s3 = boto.connect_s3(
        aws_access_key_id=keys[0],
        aws_secret_access_key=keys[1]
    )
    bucket = s3.get_bucket("dvidr", validate=False)

    k1 = Key(bucket)
    k1.key = imageFile

    url_name = k1.generate_url(259200)
    
    return url_name

filepath = (readImagefromS3("85c.jpg"))

In [55]:
# file_name = f'{path}receipt.jpg'
texts = detect_text(filepath)

In [57]:
texts

In [56]:
texts

In [41]:
simple_process(texts)

{'1 85C Coffee Bread': '$1.95',
 '1 Brioche': '$2.50',
 '1 Mango Bread': '$1.85',
 '1 Milk Butter Puff': '$SUBTOTAL',
 '1 Milk Pudding': '$3.05',
 '85 C Bakery Cafe': '$To Go'}

In [25]:
import re

def simple_process(text):
    text_list = text.split('\n')

    pattern = r'^\d+\s[\d|\D]+'
    result = {}
    for index, item in enumerate(text_list):
        if re.findall(pattern, str(item)):
            result[item] = index
    num_items = len(result.keys())
    for key in result.keys():
        result[key] = "$" + str(text_list[(result[key] + num_items)])
    return result

